In [2]:
!pip install torch torchquantum

  Using cached torchquantum-0.1.0-py2.py3-none-any.whl (33 kB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Using cached torchpack-0.3.1-py3-none-any.whl (34 kB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached torchvision-0.13.1-cp37-cp37m-manylinux1_x86_64.whl (19.1 MB)
  Using cached multimethod-1.8-py3-none-any.whl (9.8 kB)
  Using cached loguru-0.6.0-py3-none-any.whl (58 kB)
  Using cached tensorpack-0.11-py2.py3-none-any.whl (296 kB)
  Using cached msgpack_numpy-0.4.8-py2.py3-none-any.whl (6.9 kB)
  Using cached msgpack-1.0.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (299 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confl

In [4]:
!pip install pennylane

  Using cached PennyLane-0.25.1-py3-none-any.whl (1.0 MB)
  Using cached PennyLane_Lightning-0.25.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.6 MB)
  Using cached networkx-2.6.3-py3-none-any.whl (1.9 MB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached autograd-1.4-py3-none-any.whl (48 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached autoray-0.3.2-py3-none-any.whl (36 kB)
  Using cached ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (108 kB)
  Using cached future-0.18.2-py3-none-any.whl
You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.


In [5]:
import os
import numpy as np

from importlib import reload

import datasets
import create_gate_circs_np
import train_circ_np

reload(datasets)
reload(create_gate_circs_np)
reload(train_circ_np)

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'train_circ_np' from '/root/train_circ_np.py'>

In [6]:
main_dir = './experiment_data/mnist_10/trained_circuits'

inds = []

for i in range(1000):
    circ_dir = main_dir + '/circ_{}'.format(i + 1)
    
    if os.path.exists(circ_dir + '/accs.txt'):
        if np.mean(np.genfromtxt(circ_dir + '/accs.txt')) > 0.03 and not os.path.exists(circ_dir + '/accs_computed.txt'):
            inds.append(i)

In [7]:
inds

[3,
 7,
 9,
 10,
 11,
 12,
 13,
 14,
 34,
 35,
 36,
 37,
 38,
 39,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 114,
 115,
 116,
 117,
 118,
 119,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 194,
 195,
 196,
 197,
 198,
 199,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 234,
 235,
 236,
 237,
 238,
 239,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 287,
 288,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299]

In [ ]:
from datasets import TorchDataset, load_dataset
from create_gate_circs_np import get_circ_params, TQCirc, generate_random_gate_circ
from train_circ_np import train_tq_model

import pickle as pkl
import torch
import os
import numpy as np

num_qubits = 10

device = torch.device('cpu')

dataset = 'mnist_10'

train_x, train_y, test_x, test_y = load_dataset(dataset, 'angle', 1)

test_x = torch.from_numpy(test_x)
test_y = torch.from_numpy(test_y)

loss = torch.nn.functional.nll_loss
val_loss = torch.nn.NLLLoss(reduction='none')

torch.inference_mode(True)

for i in inds[3:]:
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)
    
#     if os.path.exists(curr_dir + '/accs_computed.txt'):
#         continue
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, True).to(device)
    
    model_data = torch.load(curr_dir + '/model.pt')
    
    model_params = list(model.parameters())

    for l in range(100):
        model_params[l].data.fill_(model_data['var_gates.{}.params'.format(l)].flatten().item())

    curr_val_losses = []
    curr_acc = 0
        
    for k in range(100):
        val_x, val_y = test_x[100 * k:(100 * k + 100)], test_y[100 * k:(100 * k + 100)]
        
        val_preds = model(val_x)
        curr_val_losses.append(val_loss(val_preds, val_y.to(torch.long)).detach().numpy().flatten())
        
        val_class = torch.argmax(val_preds.detach(), 1).detach().numpy().flatten()
        
        curr_acc += np.sum(val_class == val_y.detach().numpy().flatten().astype('int32'))
        
    print(i + 1, curr_acc / 10000)

    np.save(curr_dir + '/val_losses.npy', curr_val_losses)
    np.savetxt(curr_dir + '/accs.txt', [curr_acc])
    np.savetxt(curr_dir + '/accs_computed.txt', [1])

93 0.2559
153 0.3665


In [60]:
model(val_x[:1])

model.device.states[0, 0, 0, 0, 0, 0, 0, 0, 0]

tensor([[-0.0011+1.0777e-03j, -0.0012+2.1001e-04j],
        [ 0.0024-9.6106e-05j, -0.0003-7.3570e-05j]], grad_fn=<SelectBackward0>)